In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
persist_directory = "../database/anwesha_chroma_)db"

In [4]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large-instruct"
)

In [6]:
vectorstore = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

C:\Users\mdsaz\AppData\Local\Temp\ipykernel_2360\2851879184.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [7]:
retriever = vectorstore.as_retriever()

In [8]:
from langchain import hub

In [9]:
prompt = hub.pull("rlm/rag-prompt")

In [10]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

In [12]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
rag_chain.invoke("অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?")

'অনুপম তার মামার চেয়ে বছর ছয়েকের ছোট ছিল।'

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives
    | ChatGroq(model="moonshotai/kimi-k2-instruct",)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [15]:
from langchain.load import dumps, loads


def get_unique_union(documents: list[list]):
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    unique_docs = list(set(flattened_docs))
    return [loads(doc) for doc in unique_docs]


question = "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question": question})
len(docs)

7

In [16]:
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(
    model="moonshotai/kimi-k2-instruct",
)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question": question})

'অনুপম তার মামার চেয়ে **বছর ছয়েক** ছোট ছিল।'

In [17]:
sample_queries = [
    "অপরিচিতা' গল্পে, অনুপমের মতে কে আসর জমাতে অদ্বিতীয়?",
    "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?",
    "মন্দ নয় হে! খাঁটি সোনা বটে!' - এই উক্তিটি কার?",
    "কল্যাণীর বাবার নাম কী?",
    "বিবাহ-উপলক্ষ্যে কন্যাপক্ষকে কোথায় আসতে হয়েছিল?",
    "শম্ভুনাথ সেন পেশায় কী ছিলেন?",
    "অনুপম এবং তার মা কোন বাহনে তীর্থে যাচ্ছিলেন?",
    "রেলগাড়িতে কল্যাণীর সাথে কয়টি ছোট ছোট মেয়ে ছিল?",
    "বিবাহ ভাঙার পর কল্যাণী কী ব্রত গ্রহণ করে?",
    "গল্পের শেষে অনুপমের বয়স কত?"
]

In [18]:
expected_responses = []

In [19]:
for query in sample_queries:
    response = final_rag_chain.invoke({"question": query})
    expected_responses.append(response)

In [ ]:
sample_queries = [
    "অপরিচিতা' গল্পে, অনুপমের মতে কে আসর জমাতে অদ্বিতীয়?",
    "অনুপম তার মামার চেয়ে কত বছরের ছোট ছিল?",
    "মন্দ নয় হে! খাঁটি সোনা বটে!' - এই উক্তিটি কার?",
    "কল্যাণীর বাবার নাম কী?",
    "বিবাহ-উপলক্ষ্যে কন্যাপক্ষকে কোথায় আসতে হয়েছিল?",
    "শম্ভুনাথ সেন পেশায় কী ছিলেন?",
    "অনুপম এবং তার মা কোন বাহনে তীর্থে যাচ্ছিলেন?",
    "রেলগাড়িতে কল্যাণীর সাথে কয়টি ছোট ছোট মেয়ে ছিল?",
    "বিবাহ ভাঙার পর কল্যাণী কী ব্রত গ্রহণ করে?",
    "গল্পের শেষে অনুপমের বয়স কত?"
]
expected_responses = []
for query in sample_queries:
    response = final_rag_chain.invoke({"question": query})
    expected_responses.append(response)
expected_responses

['"অপরিচিতা" গল্পে, অনুপমের মতে **হরিশ** আসর জমাতে অদ্বিতীয়।',
 "'অপরিচিতা' গল্প অনুযায়ী, অনুপম তার মামার চেয়ে **বছর ছয়েক** ছোট ছিল।",
 'এই উক্তিটি **বিনুদাদার**।',
 'কল্যাণীর বাবার নাম **শম্ভুনাথ সেন**।',
 'বিবাহ-উপলক্ষ্যে কন্যাপক্ষকে **কলিকাতা** (কলকাতা) আসতে হয়েছিল।',
 'শম্ভুনাথ সেন পেশায় **উকিল** ছিলেন।',
 'উত্তর: **(গ) মোটর গাড়ি**',
 'উত্তর: ৩/৪ জন (অপশন খ)।',
 'বিবাহ ভাঙার পর কল্যাণী **অবিবাহিতা** ব্রত গ্রহণ করে।',
 'গল্পের শেষে অনুপমের বয়স **২৮ বছর**।']

In [ ]:


expected_responses = [
    "হরিশ",
    "বছর ছয়েক",
    "বিনুদা",
    "শম্ভুনাথ সেন",
    "কলিকাতা",
    "ডাক্তার",
    "রেলগাড়ি",
    "দুটি-তিনটি",
    "মেয়েদের শিক্ষার ব্রত",
    "সাতাশ"
]